In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162678 sha256=e72099b0946ee0cb6101cf2c72072c1977bd75c0d8083dcb8500958f24b9e7fd
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
from surprise import Dataset, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

In [5]:
# Завантаження датасету
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


**Пошук Найкращих Параметрів для SVD**

In [6]:
# Пошук найкращих параметрів для SVD
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

In [7]:
# Виведення кращих параметрів
best_params = gs.best_params['rmse']
print(f"Найкращі параметри для SVD: {best_params}")

Найкращі параметри для SVD: {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.1}


**Порівняння SVD, SVD++ та NMF**

In [8]:
# Створення моделей з кращими параметрами
algo_svd = SVD(**best_params)
algo_svdpp = SVDpp()
algo_nmf = NMF()

In [9]:
# Порівняння SVD з SVD++ та NMF
for model in [algo_svd, algo_svdpp, algo_nmf]:
    print(f"Evaluating {model.__class__.__name__}...")
    results = cross_validate(model, data, measures=['RMSE'], cv=3, verbose=True)
    mean_rmse = sum(results['test_rmse']) / len(results['test_rmse'])
    print(f"Середнє RMSE для {model.__class__.__name__}: {mean_rmse}\n")

Evaluating SVD...
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9290  0.9359  0.9373  0.9341  0.0036  
Fit time          2.17    2.18    2.18    2.18    0.00    
Test time         0.40    0.41    0.53    0.45    0.06    
Середнє RMSE для SVD: 0.9340623647013157

Evaluating SVDpp...
Evaluating RMSE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9200  0.9313  0.9265  0.9260  0.0047  
Fit time          19.84   19.67   19.20   19.57   0.27    
Test time         7.01    7.59    7.75    7.45    0.32    
Середнє RMSE для SVDpp: 0.9259546962312938

Evaluating NMF...
Evaluating RMSE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9716  0.9770  0.9755  0.9747  0.0023  
Fit time          2.33    2.38    2.35    2.36    0.02    
Test time         0.35    0.22    0.34    0.30    0.06    
С

**Перевірка на Перенавчання для SVD**

In [10]:
# Розділення на навчальну та тестову вибірку
trainset, testset = train_test_split(data, test_size=0.25)

# Навчання моделі SVD
algo_svd.fit(trainset)

# Передбачення на навчальній та тестовій вибірках
train_predictions = algo_svd.test(trainset.build_testset())
test_predictions = algo_svd.test(testset)

# Обчислення RMSE для обох вибірок
train_rmse = accuracy.rmse(train_predictions, verbose=False)
test_rmse = accuracy.rmse(test_predictions, verbose=False)
print(f'Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}')

Training RMSE: 0.8203391703528761, Testing RMSE: 0.9345486488355252


Висновки:

Для алгоритму SVD було проведено пошук оптимальних гіперпараметрів за допомогою GridSearchCV. Цей метод дозволив визначити найкращі параметри моделі на основі метрики RMSE.

Порівняння Моделей:

Ми зробили порівняння моделей SVD, SVD++ та NMF. Кожна модель була оцінена за допомогою крос-валідації, і було встановлено, що:

Модель SVD++ показала найкращі результати за метрикою RMSE, однак вона вимагала значно більшого часу для навчання.
Модель SVD показала трохи гірші результати за RMSE порівняно з SVD++, але забезпечила кращий баланс між точністю та часом обчислень.
Модель NMF показала найгірші результати за метрикою RMSE.

Перевірка на Перенавчання:

RMSE на Навчальній Вибірці: 0.82
RMSE на Тестовій Вибірці: 0.93

Ця різниця вказує на наявність перенавчання, хоча воно і не є значно вираженим.

Отже:
Модель SVD з використанням оптимальних гіперпараметрів показала хороший баланс між точністю передбачень і часом обчислень. Хоча SVD++ показала кращу точність, час її обчислень значно вищий. Модель NMF показала найнижчу точність за метрикою RMSE.